In [2]:
import warnings
warnings.filterwarnings("ignore")

import os, pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, glob
from pynwb import NWBHDF5IO
from pathlib import Path
from convnwb.io import load_nwbfile


## load example data file

In [3]:
# sess_file = sorted(glob.glob('../data/000623/sub-CS53*/*.nwb'))[1]
sess_file = sorted(glob.glob('../data/000623/sub-*/*.nwb'))[10]
nwbfile, io = load_nwbfile(os.path.basename(sess_file), os.path.dirname(sess_file), return_io=True)
nwbfile

,location,group,group_name,origchannel,pairwise_distances,origchannel_name,x,y,z
id,,,,,,,,,
0,Left ACC,Neuralynx-Atlas-microwire-1 pynwb.ecephys.ElectrodeGroup at 0x140346024179536\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-1,micro-1,6.7484,LACC1,-3.34,28.54,24.2
1,Left ACC,Neuralynx-Atlas-microwire-2 pynwb.ecephys.ElectrodeGroup at 0x140346024289232\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-2,micro-2,5.75438,LACC2,-3.34,28.54,24.2
2,Left ACC,Neuralynx-Atlas-microwire-3 pynwb.ecephys.ElectrodeGroup at 0x140346024168080\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-3,micro-3,NA,LACC3,-3.34,28.54,24.2
3,Left ACC,Neuralynx-Atlas-microwire-4 pynwb.ecephys.ElectrodeGroup at 0x140346024295376\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-4,micro-4,7.76186_8.26462_4.61109,LACC4,-3.34,28.54,24.2
,location,group,group_name,origchannel,pairwise_distances,origchannel_name,x,y,z
id,,,,,,,,,
0,Left ACC,Neuralynx-Atlas-microwire-1 pynwb.ecephys.ElectrodeGroup at 0x140346024179536\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-1,micro-1,6.7484,LACC1,-3.34,28.54,24.2
1,Left ACC,Neuralynx-Atlas-microwire-2 pynwb.ecephys.ElectrodeGroup at 0x140346024289232\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-2,micro-2,5.75438,LACC2,-3.34,28.54,24.2
2,Left ACC,Neuralynx-Atlas-microwire-3 pynwb.ecephys.ElectrodeGroup at 0x140346024168080\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x140346024006288\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-3,micro-3,NA,LACC3,-3.34,28.54,24.2


## helper functions

In [12]:
# choose whether to normalzie encoding FR by the trial's entire baseline or by the duration of baseline defined by shorted baseline
# norm_choice = 'min'
norm_choice = 'trial'

# Function to handle spike filtering, aligning, truncating, and FR calculation
def process_spikes(neuron, trial_df):

    trials_spikes_filtered, trials_spikes_adjusted = [], []
    trials_num_spikes, trials_RT, trials_FR = [], [], []
    trials_smooth_spikes, trials_smooth_FR = [], []
    sess_max_RT = np.max(trial_df['RT'][1:])

    for _, trial in trial_df.iterrows():
        
        # 1. Filter spike times within trial's start and stop times
        trial_spikes = neuron['spike_times'][
            (neuron['spike_times'] >= trial['start_time']) &
            (neuron['spike_times'] <= trial['stop_time'])
        ]
        
        # 2. align spike times to start & truncated to 5s
        trial_spikes_aligned = trial_spikes - trial['start_time']

        # 3. num_spikes
        trial_num_spikes = len(trial_spikes_aligned)

        # 4. firing rate
        trial_FR = len(trial_spikes_aligned)/trial['RT']

        # 5. smooth spikes

        if trial['stim_phase'] == 'encoding':
            stop, win_size, step = trial['RT'], 5, 2

        elif trial['stim_phase'] == 'baseline':
            if norm_choice == 'min':
                stop, win_size, step = 11.6, 5, 1
            elif norm_choice == 'trial':
                stop, win_size, step = trial['RT'], 5, 1
        
        elif trial['stim_phase'] == 'recognition':
            stop, win_size, step = sess_max_RT, 1, .5

        trial_smooth_spikes = []
        for i in np.arange(0, stop-win_size+step, step):
            trial_smooth_spikes.append(len(trial_spikes_aligned[(trial_spikes_aligned >= i) & (trial_spikes_aligned < i + win_size)]))
        # trial_smooth_spikes = np.array(trial_smooth_spikes)

        # 6. FR of smooth spikes
        trial_smooth_FR = np.mean(trial_smooth_spikes)
        
        # Append results to lists
        trials_spikes_filtered.append(trial_spikes)
        trials_spikes_adjusted.append(trial_spikes_aligned)
        trials_num_spikes.append(trial_num_spikes)
        trials_FR.append(trial_FR)
        trials_smooth_spikes.append(trial_smooth_spikes)
        trials_smooth_FR.append(trial_smooth_FR)

    return trials_spikes_filtered, trials_spikes_adjusted, trials_num_spikes, trials_FR, trials_smooth_spikes, trials_smooth_FR


## consolidate dataframe

In [13]:
print(norm_choice)

# skip subj1
session_files = sorted(glob.glob('../data/000623/sub-*/*.nwb'))
neuron_by_trial = []
row_counter = 0

# create table comprising n_rows = n_neurons * n_trials = ~1457 * 41
for session_file in session_files:
    
    nwbfile, io = load_nwbfile(os.path.basename(session_file), os.path.dirname(session_file), return_io=True)
    subj = os.path.basename(session_file)[:8]

    # trial data per session
    trial_df = nwbfile.trials.to_dataframe()
    
    # create a 'baseline' row
    baseline_row = {
        'start_time': trial_df['stop_time'].iloc[0],
        'stop_time': trial_df['start_time'].iloc[1],
        'stim_phase': 'baseline',
        'stimulus_file': 'NA',
        'response_correct': 'NA',
        'response_confidence': 'NA',
        'actual_response': 'NA'
    }

    # baseline row at index 1
    trial_df = pd.concat([trial_df.iloc[:1], pd.DataFrame([baseline_row]), trial_df.iloc[1:]]).reset_index(drop=True)

    trial_df['RT'] = trial_df['stop_time'] - trial_df['start_time']
    
    # all neurons within session
    neur_df = nwbfile.units.to_dataframe()[['unit_id_session', 'electrode_id', 'spike_times']]
    
    neur_df['subj'] = neur_df['unit_id_session'].str[:3]
    neur_df['sess'] = neur_df['unit_id_session'].str[:8]
    neur_df = neur_df.rename(columns={'unit_id_session': 'neuron'})

    # electrode info
    electrodes_df = nwbfile.electrodes.to_dataframe()
    
    # Replace unit_id in neur_df with the corresponding location from electrodes_df
    neur_df['electrode_id'] = neur_df['electrode_id'].map(electrodes_df['location'])
    neur_df['electrode_id'] = neur_df['electrode_id'].str.split().str[-1]
    neur_df = neur_df.rename(columns={'electrode_id': 'region'})

    for neuron in range(len(neur_df)):
    
        # get specific neuron's spike info
        neuron_row = neur_df.iloc[neuron]

        # Repeat the neuron row as many times as there are trials
        repeated_neuron_row = pd.DataFrame([neuron_row] * len(trial_df)).reset_index(drop=True)

        # Concatenate the neuron data with the trial data horizontally
        neur_tri_df = pd.concat([repeated_neuron_row, trial_df.reset_index(drop=True)], axis=1)
        
        (
            neur_tri_df['trial_spikes'],  # within trial spikes
            neur_tri_df['trial_aligned_spikes'],  # spikes aligned to start
            neur_tri_df['num_spikes'],  # number of spikes
            neur_tri_df['FR'],  # len(spikes)/RT
            neur_tri_df['trial_smooth_spikes'],  # spikes smoothed using sliding window
            neur_tri_df['trial_smooth_FR']  # FR of smoothed spikes
        ) = process_spikes(neuron_row, trial_df)

        # Finally, append the neur_tri_df for this neuron to the overall df
        neuron_by_trial.append(neur_tri_df)

    print(subj)
   
neuron_by_trial = pd.concat(neuron_by_trial, ignore_index=True)

# reorder cols
neuron_by_trial = neuron_by_trial[['subj', 'sess', 'neuron', 'region',
                                   'start_time', 'stop_time', 'RT',
                                   'trial_spikes', 'trial_aligned_spikes', 'num_spikes', 'FR',
                                   'trial_smooth_spikes', 'trial_smooth_FR',
                                   'stim_phase', 'stimulus_file',
                                   'response_correct', 'response_confidence', 'actual_response']]

neuron_by_trial.to_pickle(f'neuron_by_trial_norm_by_{norm_choice}.pkl')
neuron_by_trial.reset_index(drop=True).to_csv(f'neuron_by_trial_norm_by_{norm_choice}.csv', index=False)

    

trial
sub-CS41
sub-CS41
sub-CS42
sub-CS42
sub-CS43
sub-CS43
sub-CS44
sub-CS47
sub-CS47
sub-CS48
sub-CS48
sub-CS49
sub-CS49
sub-CS51
sub-CS51
sub-CS53
sub-CS53
sub-CS54
sub-CS54
sub-CS55
sub-CS55
sub-CS56
sub-CS56
sub-CS57
sub-CS57
sub-CS58
sub-CS60
sub-CS62
sub-CS62


In [14]:
neuron_by_trial.head()

,subj,sess,neuron,region,start_time,stop_time,RT,trial_spikes,trial_aligned_spikes,num_spikes,FR,trial_smooth_spikes,trial_smooth_FR,stim_phase,stimulus_file,response_correct,response_confidence,actual_response
0,P41,P41CS_R1,P41CS_R1_21_1_679_4,amygdala,0.000000,478.855120,478.855120,"[3.43207675, 4.39507675, 5.16801325, 5.2624507...","[3.43207675, 4.39507675, 5.16801325, 5.2624507...",360,0.751793,"[2, 5, 8, 9, 5, 4, 1, 2, 3, 3, 4, 3, 4, 4, 4, ...",3.743697,encoding,bd_movie,NaN,NaN,NaN
1,P41,P41CS_R1,P41CS_R1_21_1_679_4,amygdala,478.855120,566.346460,87.491340,"[480.126667, 482.70775975, 483.49129, 488.6475...","[1.2715469999999982, 3.8526397499999803, 4.636...",50,0.571485,"[3, 3, 2, 2, 1, 1, 1, 1, 4, 5, 6, 6, 8, 5, 4, ...",2.904762,baseline,NA,NA,NA,NA
2,P41,P41CS_R1,P41CS_R1_21_1_679_4,amygdala,566.346460,577.078362,10.731902,"[566.79785075, 568.845881, 569.40153725, 576.0...","[0.4513907499999732, 2.499420999999984, 3.0550...",4,0.372721,"[1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.040230,recognition,old015.png,1.0,2.0,5.0
3,P41,P41CS_R1,P41CS_R1_21_1_679_4,amygdala,578.122799,582.712578,4.589779,[582.05028225],[3.927483250000023],1,0.217875,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.011494,recognition,new021.png,1.0,3.0,1.0
4,P41,P41CS_R1,P41CS_R1_21_1_679_4,amygdala,583.755203,591.685106,7.929903,[589.2551855],[5.499982499999987],1,0.126105,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",0.011494,recognition,new030.png,0.0,3.0,6.0
